In [2]:
import numpy as np
import pandas as pd
import pyflux as pf
import datetime as datetime
import matplotlib.pyplot as plt
import os
import pickle
import ForecastModel as fc

In [3]:
X_fwds = pickle.load(open('../data/forward_rates', 'rb'))
#X_fwds = df_add_first_diff(X_fwds)

#df_FX = pickle.load( open( "data/FX_data", "rb" ) )
# Loading up the federal reserve speech data
fed_metrics = pickle.load( open( "../data/final_press_rel_dist", "rb" ) )

In [5]:
cos_last = fed_metrics['cos_last']
cos_avg_n = fed_metrics['cos_avg_n']
ed_last = fed_metrics['ed_last']
ed_avg_n = fed_metrics['ed_avg_n']
fed_dates = fed_metrics['dates']

#grouping by date (some dates had multiple speeches)
avgstats = pd.DataFrame({'date':fed_dates,
                        'ed_last': ed_last,
                        'ed_avg_n': ed_avg_n,
                        'cos_last': cos_last,
                        'cos_avg_n': cos_avg_n}).groupby('date').mean()
avgstats.index = pd.to_datetime(avgstats.index)

In [6]:
this_min = min(avgstats.index)
this_min

Timestamp('2014-09-17 00:00:00')

In [8]:
X_fwds.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3313 entries, 2006-01-03 to 2019-03-28
Data columns (total 14 columns):
six_m        3313 non-null float64
one_y        3313 non-null float64
two_y        3313 non-null float64
three_y      3313 non-null float64
five_y       3313 non-null float64
seven_y      3313 non-null float64
ten_y        3313 non-null float64
d_six_m      3312 non-null float64
d_one_y      3312 non-null float64
d_two_y      3312 non-null float64
d_three_y    3312 non-null float64
d_five_y     3312 non-null float64
d_seven_y    3312 non-null float64
d_ten_y      3312 non-null float64
dtypes: float64(14)
memory usage: 388.2 KB


In [9]:
# now I need to filter the X_fwds to drop anything before this period!
X_after = X_fwds[X_fwds.index >= this_min]

In [11]:
X_fwds = X_fwds[X_fwds.index >= this_min]
X_fwds = X_fwds.merge(avgstats, how='left', left_index = True, right_index = True)
X_fwds.fillna(value=0, inplace=True)

# the first row of X_fwds contains zeros for the differenced rates, clear them here
X_fwds = X_fwds.drop(X_fwds.index[0])


In [13]:
X_fwds.describe()

,six_m,one_y,two_y,three_y,five_y,seven_y,ten_y,d_six_m,d_one_y,d_two_y,d_three_y,d_five_y,d_seven_y,d_ten_y,ed_last,ed_avg_n,cos_last,cos_avg_n
count,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1.131000e+03,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000
mean,0.009853,0.012818,0.016338,0.019881,0.024100,0.028463,0.027897,0.000021,0.000019,0.000009,5.756270e-08,-0.000007,-0.000009,-0.000008,0.014717,0.014717,0.028184,0.028184
std,0.008479,0.008678,0.007219,0.005697,0.004369,0.003448,0.003372,0.000180,0.000346,0.000533,5.923067e-04,0.000574,0.000615,0.000508,0.082246,0.082246,0.155637,0.155637
min,0.000300,0.001401,0.006012,0.008720,0.013098,0.018210,0.018246,-0.000800,-0.001924,-0.002230,-2.011883e-03,-0.002007,-0.002510,-0.001964,0.000000,0.000000,0.000000,0.000000
25%,0.002700,0.005609,0.010331,0.016127,0.021580,0.026422,0.026148,-0.000100,-0.000201,-0.000303,-4.045139e-04,-0.000381,-0.000405,-0.000327,0.000000,0.000000,0.000000,0.000000
50%,0.006200,0.009927,0.014452,0.018891,0.024478,0.028839,0.028559,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-0.000024,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.017300,0.021112,0.023588,0.023992,0.027050,0.031127,0.030298,0.000100,0.000201,0.000305,4.039032e-04,0.000355,0.000393,0.000312,0.000000,0.000000,0.000000,0.000000
max,0.025800,0.029923,0.032666,0.032461,0.032640,0.036281,0.034891,0.001000,0.001708,0.002012,2.221905e-03,0.002655,0.003046,0.002071,0.621916,0.621916,0.943415,0.943415


In [34]:
temp = X_fwds['ed_last'].copy()

In [37]:
temp =pd.concat([temp, temp.shift(), temp.shift(2)], axis=1)

In [39]:
temp.head()

,ed_last,ed_last,ed_last
Date,,,
2014-09-18,0.0,NaN,NaN
2014-09-19,0.0,0.0,NaN
2014-09-22,0.0,0.0,0.0
2014-09-23,0.0,0.0,0.0
2014-09-24,0.0,0.0,0.0


In [43]:
X_fwds['ed_last_l1']= temp.iloc[:,1]
X_fwds['ed_last_l2']= temp.iloc[:,2]
X_fwds.head()

,six_m,one_y,two_y,three_y,five_y,seven_y,ten_y,d_six_m,d_one_y,d_two_y,d_three_y,d_five_y,d_seven_y,d_ten_y,ed_last,ed_avg_n,cos_last,cos_avg_n,ed_last_l1,ed_last_l2
Date,,,,,,,,,,,,,,,,,,,,
2014-09-18,0.0004,0.002001,0.010841,0.021453,0.030432,0.036281,0.034781,-0.0001,0.0001,0.000000,0.000609,0.000484,0.000316,-0.000434,0.0,0.0,0.0,0.0,NaN,NaN
2014-09-19,0.0004,0.001801,0.010942,0.021148,0.030050,0.035673,0.034115,0.0000,-0.0002,0.000101,-0.000305,-0.000382,-0.000608,-0.000666,0.0,0.0,0.0,0.0,0.0,NaN
2014-09-22,0.0004,0.001601,0.010841,0.020438,0.029746,0.035367,0.034150,0.0000,-0.0002,-0.000101,-0.000710,-0.000304,-0.000305,0.000035,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-23,0.0004,0.001601,0.010640,0.020337,0.029364,0.034760,0.033859,0.0000,0.0000,-0.000202,-0.000101,-0.000382,-0.000608,-0.000291,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-24,0.0003,0.001901,0.010942,0.020843,0.029949,0.034819,0.034140,-0.0001,0.0003,0.000302,0.000507,0.000585,0.000059,0.000281,0.0,0.0,0.0,0.0,0.0,0.0


,six_m,one_y,two_y,three_y,five_y,seven_y,ten_y,d_six_m,d_one_y,d_two_y,d_three_y,d_five_y,d_seven_y,d_ten_y,ed_last,ed_avg_n,cos_last,cos_avg_n,ed_last_l1,ed_last_l2
Date,,,,,,,,,,,,,,,,,,,,
2014-09-22,0.0004,0.001601,0.010841,0.020438,0.029746,0.035367,0.034150,0.0000,-0.0002,-0.000101,-0.000710,-0.000304,-0.000305,0.000035,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-23,0.0004,0.001601,0.010640,0.020337,0.029364,0.034760,0.033859,0.0000,0.0000,-0.000202,-0.000101,-0.000382,-0.000608,-0.000291,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-24,0.0003,0.001901,0.010942,0.020843,0.029949,0.034819,0.034140,-0.0001,0.0003,0.000302,0.000507,0.000585,0.000059,0.000281,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-25,0.0003,0.001701,0.010438,0.019931,0.028906,0.034813,0.033656,0.0000,-0.0002,-0.000504,-0.000912,-0.001043,-0.000006,-0.000485,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-26,0.0003,0.001901,0.010942,0.020843,0.029413,0.034605,0.033586,0.0000,0.0002,0.000504,0.000912,0.000507,-0.000208,-0.000069,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
temp2

array([0., 0., 0., ..., 0., 0., 0.])

In [30]:
len(temp2)

1130

In [32]:
len(temp)

1131